In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import datetime
import gc
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,KFold, GridSearchCV
from sklearn.metrics import mean_squared_error


import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb

%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso,LogisticRegression)
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    

    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:

def missing_statistics(df):    
    statitics = pd.DataFrame(df.isnull().sum()).reset_index()
    statitics.columns=['COLUMN NAME',"MISSING VALUES"]
    statitics['TOTAL ROWS'] = df.shape[0]
    statitics['% MISSING'] = round((statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100,2)
    return statitics

In [ ]:
df_train = import_data('/kaggle/input/testval-processing/df_train.csv')

In [ ]:
df_val = import_data('/kaggle/input/testval-processing/df_val.csv')

In [ ]:
data_test = import_data('/kaggle/input/testval-processing/data_test.csv')
data_test["datetime"] = pd.to_datetime(data_test["timestamp"])
data_test["week"] = data_test["datetime"].dt.week;

# LGBM Model

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_train.primary_use = le.fit_transform(df_train.primary_use)
df_val.primary_use = le.fit_transform(df_val.primary_use)
data_test.primary_use = le.fit_transform(data_test.primary_use)

Make validation set based on time split

In [ ]:
df_val.shape[0] #25% of train data for validation

In [ ]:
df_train.shape[0] #75% of train data for the training

In [ ]:
y_train = df_train['meter_reading']
y_val = df_val['meter_reading']


df_train = df_train.drop(columns=['meter_reading'])
df_val = df_val.drop(columns=['meter_reading'])

In [ ]:
gc.collect()

In [ ]:
# Drop timestamp because model does not accept
df_train = df_train.drop(columns=['timestamp'])
df_val = df_val.drop(columns=['timestamp'])
data_test = data_test.drop(columns=['timestamp'])


In [ ]:
from lightgbm import LGBMRegressor,LGBMClassifier, plot_importance
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit
# Model
params = {
    'application': 'regression', # for regression
#     'num_class' : 1, # used for multi-classes
    'boosting': 'gbdt', # traditional gradient boosting decision tree
    'num_iterations': 100, 
    'learning_rate': 0.05,
    'num_leaves': 62,
    'device': 'cpu', # you can use GPU to achieve faster learning
    'max_depth': -1, # <0 means no limit
    'max_bin': 510, # Small number of bins may reduce training accuracy but can deal with over-fitting
    'lambda_l1': 5, # L1 regularization
    'lambda_l2': 10, # L2 regularization
    #'metric' : 'binary_error',
    'subsample_for_bin': 200, # number of samples for constructing bins
    'subsample': 1, # subsample ratio of the training instance
    'colsample_bytree': 0.8, # subsample ratio of columns when constructing the tree
    'min_split_gain': 0.5, # minimum loss reduction required to make further partition on a leaf node of the tree
    'min_child_weight': 1, # minimum sum of instance weight (hessian) needed in a leaf
    'min_child_samples': 5# minimum number of data needed in a leaf
}

# Initiate classifier to use
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt', 
          objective = 'regression', 
          n_jobs = 5, 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'])

# To view the default model parameters:
mdl.get_params().keys()

**Grid Search**

In [ ]:
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import mean_squared_error
gridParams = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [8,16,24],
    'num_leaves': [6,8,12,16], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['regression'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [500],
    'colsample_bytree' : [0.64, 0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

grid = GridSearchCV(mdl, gridParams, verbose=1, cv=5, n_jobs= 1, scoring='neg_mean_squared_error')
# Run the grid
grid.fit(df_train, y_train)



# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)
# Root Mean Squared Error
print(np.sqrt(-model.best_score_))

Use best model

In [ ]:
params['colsample_bytree'] = grid.best_params_['colsample_bytree']
params['learning_rate'] = grid.best_params_['learning_rate'] 
params['max_bin'] = grid.best_params_['max_bin']
params['num_leaves'] = grid.best_params_['num_leaves']
params['reg_alpha'] = grid.best_params_['reg_alpha']
params['reg_lambda'] = grid.best_params_['reg_lambda']
params['subsample'] = grid.best_params_['subsample']



d_train = lgb.Dataset(df_train, label=y_train)
d_valid = lgb.Dataset(df_val, label=y_val) 

watchlist = [d_train, d_valid]


model = lgb.train(params, train_set=d_train, num_boost_round=1000, valid_sets=watchlist, early_stopping_rounds=50, verbose_eval=4)

y_predict = model.predict(df_val)
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
mse = mse(y_val, y_predict)
print('MSE: %.3f, RMSE: %.3f ' % (mse, sqrt(mse)))

In [ ]:
df_val.head()

In [ ]:
data_test.head()

In [ ]:
data_test = data_test.drop(columns=['datetime','row_id'])
data_test.head()

In [ ]:
STEP = 1000000
y_test_predict = []
for i in range(0, data_test.shape[0], STEP):
    batch_prediction = model.predict(data_test.loc[i:i+STEP-1,:])
    #batch_prediction = lgbmr.predict(data_test.loc[i:i+STEP-1,:])
    y_test_predict.append(list(batch_prediction))
y_test = []
for predictions in y_test_predict:
    y_test = y_test + predictions
y_test = np.array(y_test)

# submition data

In [ ]:
submission=pd.read_csv('/kaggle/input/ashrae-energy-prediction/sample_submission.csv')# Le fichier sample_submission.csv est inclut dans les fichiers de la compétition et contient le template de soumission

data_test['meter_reading']=y_test #Ajouter les prédictions dans la base de données test
data_to_convtest=data_test[data_test['site_id']==0].index #Convertir les prédictions du site_id  0 
for j in data_to_convtest :
    data_test.at[j,'meter_reading']=(data_test.at[j,'meter_reading']*3.4118)
submission['meter_reading'] = data_test['meter_reading'] 
submission['meter_reading'].clip(lower=0,upper=None,inplace=True) # au cas ou le modèle a prédit une valeur négative
submission.to_csv("Advanced_EDA_LGBM.csv",index=None) #cette fonction va générer le fichier de soumission avec les prédictions 
submission.head(5)